# Requirements

In [2]:
import json
import numpy as np
import re
import pandas as pd
import geopandas as gpd
import nltk
pd.options.display.max_columns = 1000 # to see all columns
import warnings
warnings.filterwarnings('ignore')
import sddk

# Loading datasets

In [15]:
# find local version of the input dataset
local_paths = !find ~/Projects -name "LIREg.parquet"
print(local_paths)

['/Users/kasev/Projects/LIRE_ETL/data/large_data/LIREg.parquet']


In [16]:
# read LIRE dataset from zenodo or locally, if already available
try:
    LIRE = gpd.read_parquet(local_paths[0])
    print("local copy found and loaded")
except:# !!! use carefully - takes time
    q = input("local file not found, try another put, or input 'y' for download")
    if q == "y":
        LIRE = gpd.read_file("https://zenodo.org/record/5074774/files/LIREg.geojson?download=1", driver="geoJSON")
        print("file downloaded from zenodo")
        LIRE.to_parquet("../data/large_data/LIREg.parquet")

local copy found and loaded


In [17]:
LIRE.head(5)

,EDCS-ID,publication,province,province_list,place,place_list,end_yr_list,notes_dating,status_list,inscr_type,status_notation,inscr_process,notes_references,notes_comments,inscription,inscription_stripped_final,Links,dating from,dating to,status,Latitude,Longitude,photo,Material,Comment,EDH-ID,language_EDCS,clean_text_interpretive_word_EDCS,responsible_individual,type_of_inscription,letter_size,literature,work_status,height,diplomatic_text,people,depth,material,type_of_monument,province_label,width,transcription,country,uri,findspot_ancient,last_update,modern_region,findspot_modern,edh_geography_uri,commentary,trismegistos_uri,external_image_uris,fotos,idno_tm,placenames_refs,text_edition,origdate_text,layout_execution,layout_execution_text,support_objecttype,support_objecttype_text,support_material,support_material_text,support_decoration,keywords_term,keywords_term_text,type_of_inscription_clean,type_of_inscription_certainty,height_cm,width_cm,depth_cm,material_clean,type_of_monument_clean,type_of_monument_certainty,province_label_clean,province_label_certainty,country_clean,country_certainty,findspot_ancient_clean,findspot_ancient_certainty,modern_region_clean,modern_region_certainty,findspot_modern_clean,findspot_modern_certainty,findspot_clean,findspot_certainty,origdate_text_clean,clean_text_conservative,clean_text_interpretive_sentence,findspot,year_of_find,present_location,religion,geography,social_economic_legal_history,military,not_after,language,not_before,coordinates,clean_text_interpretive_word,urban_context,within_rome,nearest_city,city_id_hanson,city_pop_est,city_geometry,nearest_city_type,nearest_city_dist,type_of_inscription_auto,type_of_inscription_auto_prob,geometry
0,EDCS-03700724,"ZPE-108-159 = Thesprotia 00001 = AE 1993, 0140...",Achaia,Achaia,Agios Athanasios / Photike,"['Agios Athanasios', 'Photike']",313,,"['Augusti/Augustae', 'litterae erasae', 'ordo ...",tituli honorarii,"['Augusti/Augustae', 'ordo equester', 'tria no...",litterae erasae,,,Fortissimo et Piis/simo Caesari d(omino) n(ost...,Fortissimo et Piis/simo Caesari d(omino) n(ost...,http://db.edcs.eu/epigr/partner.php?s_language...,309.0,313,Augusti/Augustae; litterae erasae; ordo equest...,39.451218,20.766767,http://db.edcs.eu/epigr/bilder.php?bilder.php?...,None,None,HD052964,None,Fortissimo et Piissimo Caesari domino nostro G...,Cowey,honorific inscription,3-5.3 cm,"AE 1993, 1406.; V. Papadopoulou, AD 43 B, 1988...",checked with photo,99 cm,FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...,"[{'name': 'Gal. Val. [[Maximiano]]', 'nomen': ...",67 cm,None,statue base,Epirus,67 cm,Fortissimo et piis/simo Caesari d(omino) n(ost...,Greece,https://edh-www.adw.uni-heidelberg.de/edh/insc...,Photike,2014-09-16,Ípeiros,Paramythía,https://edh-www.adw.uni-heidelberg.de/edh/geog...,None,https://www.trismegistos.org/text/121715,{ },{ },121715,"['http://www.trismegistos.org/place/000655', '...",Fortissimo et piissimo Caesari...,309 AD – 313 AD,21,unbestimmt,57.0,Statuenbasis,138,unbestimmt,1000.0,69,Ehreninschrift,honorific inscription,Certain,99.0,67.0,67.0,NULL,statue base,Certain,Epirus,Certain,Greece,Certain,Photike,Certain,Ípeiros,Certain,Paramythía,Certain,Agios Athanasios,Certain,309 AD – 313 AD,Fortissimo et piissimo Caesari d n Gal Val P F...,Fortissimo et piissimo Caesari domino nostro G...,Agios Athanasios,None,Fragma Kalama,None,data available,None,None,313.0,Latin,309.0,"[20.7668, 39.4512]",Fortissimo et piissimo Caesari domino nostro G...,rest,False,Dodona,31,1000.0,"[20.787767, 39.546432]",minor,0.097513,honorific inscription,1.0,POINT (20.76680 39.45120)
1,EDCS-03300852,"AE 1995, 01409",Achaia,Achaia,Alea / Tegea,"['Alea', 'Tegea']",276,to 276; b: 276 to 282 \n\n,"['Augusti/Augustae', 'miliaria', 'viri']",miliaria,"['Augusti/Augustae', 'viri']",{ },,,to 276; b: 276 to 282 \n\n \n \nImp...,Imp(eratori) / Floriano / P(io) F(elici) Aug(u...,http://db.edcs.eu/epigr/partner.php?s_language...,NaN,276,Augusti/Augustae; miliaria; viri,37.454501,22.4208

In [18]:
print(LIRE.columns)

Index(['EDCS-ID', 'publication', 'province', 'province_list', 'place',
       'place_list', 'end_yr_list', 'notes_dating', 'status_list',
       'inscr_type',
       ...
       'within_rome', 'nearest_city', 'city_id_hanson', 'city_pop_est',
       'city_geometry', 'nearest_city_type', 'nearest_city_dist',
       'type_of_inscription_auto', 'type_of_inscription_auto_prob',
       'geometry'],
      dtype='object', length=112)


# Function to extract occupations

In [26]:
occups_declined_dict = json.load(open("../data/occups_declined_dict.json"))

In [27]:
def extract_occup(inscription_text):
    occups_found = []
    if not isinstance(inscription_text, str): # if not valid string
        inscription_text = ""
    for occup in occups_declined_dict.keys():
        for occup_morph in occups_declined_dict[occup]:
            try:
                if occup_morph in inscription_text: # first check it this way, otherwise skip
                    occup_morph_N = len(re.findall("(\W|^)" + occup_morph + "(\W|$)", inscription_text))
                    if occup_morph_N > 0:
                        occups_found.extend([occup] * occup_morph_N)
                        inscription_text = re.sub("(\W|^)(" + occup_morph + ")(\W|$)", r"\1", inscription_text)
            except: pass
    return occups_found

In [28]:
extract_occup("curatores, procuratores et negotiatores curatori navium et curatori")

['curator navium', 'negotiator', 'curator', 'curator']

# Occupations - extraction

In [29]:
# check that our occupations are properly arranged (from the longest...)
list(occups_declined_dict.keys())[:20]

['exactor auri argenti et aeris',
 'inclusor auri et gemmarum',
 'tesserarius lignarius',
 'refector pectinarius',
 'instructor parietum',
 'tribor argentarius',
 'carrocarpentarius',
 'dactilidiogliphus',
 'manceps salinarum',
 'manceps thermarum',
 'aequator monetae',
 'conchyliolegulus',
 'orchestopolarius',
 'rhypararographus',
 'calciamentarius',
 'chorographarius',
 'diffusor oleari',
 'domnicomontanus',
 'faber tignuarii',
 'medicamentarius']

In [30]:
%%time
LIRE["occups"] = LIRE["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 1min 3s, sys: 248 ms, total: 1min 3s
Wall time: 1min 3s


In [31]:
LIRE["occups_N"] = LIRE["occups"].apply(len)

In [32]:
LIRE["occups_N"].sum() # previously 3840

5244

In [33]:
LIRE_occups_list = [el for sublist in LIRE["occups"].tolist() for el in sublist]
occupations_counts = pd.DataFrame(nltk.FreqDist(LIRE_occups_list).most_common(), columns=["occupation", "count"])
occupations_counts.head(10)

,occupation,count
0,curator,1069
1,faber,690
2,medicus,252
3,aerarius,237
4,scriba,224
5,negotiator,148
6,sagittarius,128
7,centonarius,124
8,mensor,85
9,argentarius,80


In [34]:
# load the occupation list with all metadata
occupations_df = pd.read_csv("../data/occupations_list_hisco.csv")
occupations_df.head(5)

,Term,gen_sg,Vocab_nom_sg,Source,HISCO_majorgroup,HISCO_minorgroup,Harris_Category,Subcategory,Translation_eng
0,abetarius,i,NaN,Petrikovits 1981a,8.0,81.0,Building,Wood worker,"a joiner, wood worker"
1,abietarius,i,NaN,Petrikovits 1981a,8.0,81.0,Building,Wood worker,"a joiner, wood worker"
2,acceptor,oris,acceptor,Waltzing - Rome,3.0,31.0,Finance,NaN,"collector, gold quality checker"
3,accomodator,oris,NaN,Petrikovits 1981a,9.0,99.0,Unclear meaning,NaN,"uncertain, craftsman"
4,aceptor,oris,NaN,Petrikovits 1981a,3.0,31.0,Finance,NaN,"collector, gold quality checker"


In [35]:
def get_int(x):
    try: return str(int(x))
    except: return ""
occupations_df["HISCO_majorgroup"] = occupations_df["HISCO_majorgroup"].apply(get_int)
occupations_df["HISCO_minorgroup"] = occupations_df["HISCO_minorgroup"].apply(get_int)

In [36]:
occupations_df.head(5)

,Term,gen_sg,Vocab_nom_sg,Source,HISCO_majorgroup,HISCO_minorgroup,Harris_Category,Subcategory,Translation_eng
0,abetarius,i,NaN,Petrikovits 1981a,8,81,Building,Wood worker,"a joiner, wood worker"
1,abietarius,i,NaN,Petrikovits 1981a,8,81,Building,Wood worker,"a joiner, wood worker"
2,acceptor,oris,acceptor,Waltzing - Rome,3,31,Finance,NaN,"collector, gold quality checker"
3,accomodator,oris,NaN,Petrikovits 1981a,9,99,Unclear meaning,NaN,"uncertain, craftsman"
4,aceptor,oris,NaN,Petrikovits 1981a,3,31,Finance,NaN,"collector, gold quality checker"


In [37]:
occupation_dict = {}
keys = ["Harris_Category", "Source", "HISCO_majorgroup", "Subcategory", "HISCO_minorgroup", "Translation_eng"]
for n in range(len(occupations_df)):
    occupation_dict[occupations_df.iloc[n]["Term"]] = dict([(key, occupations_df.iloc[n][key]) for key in keys])

In [38]:
occupation_dict

{'abetarius': {'Harris_Category': 'Building',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '8',
  'Subcategory': 'Wood worker',
  'HISCO_minorgroup': '81',
  'Translation_eng': 'a joiner, wood worker'},
 'abietarius': {'Harris_Category': 'Building',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '8',
  'Subcategory': 'Wood worker',
  'HISCO_minorgroup': '81',
  'Translation_eng': 'a joiner, wood worker'},
 'acceptor': {'Harris_Category': 'Finance',
  'Source': 'Waltzing - Rome',
  'HISCO_majorgroup': '3',
  'Subcategory': nan,
  'HISCO_minorgroup': '31',
  'Translation_eng': 'collector, gold quality checker'},
 'accomodator': {'Harris_Category': 'Unclear meaning',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '9',
  'Subcategory': nan,
  'HISCO_minorgroup': '99',
  'Translation_eng': 'uncertain, craftsman'},
 'aceptor': {'Harris_Category': 'Finance',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '3',
  'Subcategory': nan,
  'HISCO_minorgroup': '31',


In [41]:
for key in keys:
    occupations_counts[key] = occupations_counts["occupation"].apply(lambda x: occupation_dict[x][key])
occupations_counts.head(10)

,occupation,count,Harris_Category,Source,HISCO_majorgroup,Subcategory,HISCO_minorgroup,Translation_eng
0,curator,1069,Managerial,Waltzing,2,NaN,21,"he who takes charge, a manager, overseer, supe..."
1,faber,690,Metal-Working,Waltzing - Rome,9,NaN,99,"a worker in wood, stone, metal, etc., a forger..."
2,medicus,252,Miscellaneous Services,Waltzing - Rome,0,NaN,6,a surgeon
3,aerarius,237,Metal-Working,Waltzing - Rome,8,NaN,83,metal worker
4,scriba,224,Education,Waltzing - Rome,3,NaN,30,"a public writer, official scribe, professional..."
5,negotiator,148,Retail,Waltzing - Rome,4,NaN,43,"one who does business by wholesale, a wholesal..."
6,sagittarius,128,Metal-Working,Petrikovits 1981a,8,NaN,83,"arrow-makers, arrow-smiths"
7,centonarius,124,Clothing,Waltzing - Rome,7,NaN,79,"a maker of patchwork, a dealer in rags"
8,mensor,85,Miscellaneous Services,Waltzing - Rome,0,NaN,2,"a measurer, surveyor"
9,argentarius,80,Metal-Working,Waltzing - Rome,8,NaN,88,"jewellery maker, banker"


In [39]:
HISCO_majorgroup_dict = {
    "0" : "Professional, technical and related workers",
    "1" : "Professional, technical and related workers",
    "2" : "Administrative and managerial workers",
    "3" : "Clerical and related workers",
    "4" : "Sales workers",
    "5" : "Service workers",
    "6" : "Agricultural, animal husbandry and forestry workers, fishermen and hunters",
    "7" : "Production and related workers, transport equipment operators and labourers",
    "8" : "Production and related workers, transport equipment operators and labourers",
    "9" : "Production and related workers, transport equipment operators and labourers",
    "" : ""
}

In [42]:
occupations_counts["HISCO_majorgroup_descr"] =  occupations_counts["HISCO_majorgroup"].apply(lambda x: HISCO_majorgroup_dict[x])
occupations_counts.head(5)

,occupation,count,Harris_Category,Source,HISCO_majorgroup,Subcategory,HISCO_minorgroup,Translation_eng,HISCO_majorgroup_descr
0,curator,1069,Managerial,Waltzing,2,NaN,21,"he who takes charge, a manager, overseer, supe...",Administrative and managerial workers
1,faber,690,Metal-Working,Waltzing - Rome,9,NaN,99,"a worker in wood, stone, metal, etc., a forger...","Production and related workers, transport equi..."
2,medicus,252,Miscellaneous Services,Waltzing - Rome,0,NaN,6,a surgeon,"Professional, technical and related workers"
3,aerarius,237,Metal-Working,Waltzing - Rome,8,NaN,83,metal worker,"Production and related workers, transport equi..."
4,scriba,224,Education,Waltzing - Rome,3,NaN,30,"a public writer, official scribe, professional...",Clerical and related workers


In [43]:
occupations_counts.to_csv("../data/occupations_counts.csv")

## Occupations - overview

In [180]:
# EDH
LIRE[LIRE["EDH-ID"].notnull()]["occups_N"].sum() # 1272

1798

In [181]:
# EDCS
LIRE[LIRE["EDH-ID"].isnull()]["occups_N"].sum() # 2568

3446

In [182]:
print("LIRE - number of occupation occurances: " + str(LIRE["occups_N"].sum()))
print("LIRE - number of inscriptions with at least one occupation mentioned: " + str(len(LIRE[LIRE["occups_N"] > 0])))

LIRE - number of occupation occurances: 5244
LIRE - number of inscriptions with at least one occupation mentioned: 4088


In [183]:
occupations_counts.groupby("HISCO_majorgroup_descr").sum()

,count
HISCO_majorgroup_descr,
,4
Administrative and managerial workers,1167
"Agricultural, animal husbandry and forestry workers, fishermen and hunters",113
Clerical and related workers,331
"Production and related workers, transport equipment operators and labourers",2206
"Professional, technical and related workers",700
Sales workers,406
Service workers,317


In [184]:
occupations_counts.groupby("HISCO_majorgroup").sum()

,count
HISCO_majorgroup,
,4
0,448
1,252
2,1167
3,331
4,406
5,317
6,113
7,354


# Organizations - extraction

In [44]:
# read declined occupations
orgs_declined_dict = json.load(open("../data/organizations_declined_dict.json", "r"))

In [45]:
# check that our occupations are properly arranged (from the longest...)
list(orgs_declined_dict.keys())[:15]

['collegatarius',
 'collegiarius',
 'collegiatus',
 'corporatus',
 'sodalicium',
 'collegium',
 'collegius',
 'concilium',
 'conlegium',
 'sodalitas',
 'colegium',
 'sociatio',
 'societas',
 'collega',
 'corpus']

In [46]:
def extract_orgs(inscription_text):
    orgs_found = []
    if not isinstance(inscription_text, str): # if not valid string
        inscription_text = ""
    for org in orgs_declined_dict.keys():
        for orgs_morph in orgs_declined_dict[org]:
            try:
                if orgs_morph in inscription_text: # first check it this way, otherwise skip
                    orgs_morph_N = len(re.findall("(\W|^)" + orgs_morph + "(\W|$)", inscription_text))
                    if orgs_morph_N > 0:
                        orgs_found.extend([org] * orgs_morph_N)
                        inscription_text = re.sub("(\W|^)(" + orgs_morph + ")(\W|$)", r"\1", inscription_text)
            except: pass
    return orgs_found

In [47]:
%%time
LIRE["organizations"] = LIRE["clean_text_interpretive_word"].apply(extract_orgs)

CPU times: user 1.36 s, sys: 7.94 ms, total: 1.36 s
Wall time: 1.38 s


In [48]:
LIRE["organizations_N"] = LIRE["organizations"].apply(len)

In [49]:
LIRE_organizations_list = [el for sublist in LIRE["organizations"].tolist() for el in sublist]
print(len(LIRE_organizations_list)) # previously 2949 - we had a lot of "collegius"...
print(nltk.FreqDist(LIRE_organizations_list).most_common(30))

1812
[('collegium', 1152), ('corpus', 388), ('collega', 111), ('colegium', 51), ('collegius', 25), ('sodalicium', 17), ('corporatus', 16), ('collegiatus', 14), ('conlegium', 13), ('societas', 12), ('concilium', 12), ('collegiarius', 1)]


In [50]:
organizations_counts = pd.DataFrame(nltk.FreqDist(LIRE_organizations_list).most_common(), columns=["term", "count"])
organizations_counts

,term,count
0,collegium,1152
1,corpus,388
2,collega,111
3,colegium,51
4,collegius,25
5,sodalicium,17
6,corporatus,16
7,collegiatus,14
8,conlegium,13
9,societas,12


## Organizations - overview

In [51]:
# EDH
LIRE[LIRE["EDH-ID"].notnull()]["organizations_N"].sum()

578

In [52]:
# EDCS
LIRE[LIRE["EDH-ID"].isnull()]["organizations_N"].sum()

1234

In [53]:
organizations_counts.to_csv("../data/organizations_counts.csv")

In [54]:
print("LIRE - number of organization occurances: " + str(LIRE["organizations_N"].sum()))
print("LIRE - number of inscriptions with at least one organization mentioned: " + str(len(LIRE[LIRE["organizations_N"] > 0])))

LIRE - number of organization occurances: 1812
LIRE - number of inscriptions with at least one organization mentioned: 1242


# Saving

In [56]:
LIRE.to_parquet("../data/large_data/LIREg_occupsorgs.parquet")

In [196]:
# save locally
with open("../data/large_data/LIREg_occupsorgs.geojson", "w") as file: 
    json.dump(sddk.gdf_to_geojson(LIRE), file)

In [57]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [58]:
s.write_file("SDAM_data/social_diversity/LIRE_occupsorgs_2021-11-26.geojson", LIRE)

Your <class 'geopandas.geodataframe.GeoDataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/social_diversity/LIRE_occupsorgs_2021-11-26.geojson"
